In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv',delimiter=',',unpack=True,dtype='float32')
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

C:\Users\hansh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hansh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hansh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hansh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

In [5]:
global_step = tf.Variable(0,trainable = False, name='global_step')
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2,10],-1.,1.),name='W1')
    L1 = tf.nn.relu(tf.matmul(X,W1))
    tf.summary.histogram("Weights1",W1)

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10,20],-1.,1.),name='W2')
    L2 = tf.nn.relu(tf.matmul(L1,W2))
    tf.summary.histogram("Weights2",W2)

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20,3],-1.,1.),name='W3')
    model = tf.matmul(L2,W3)
    tf.summary.histogram("Weights3",W3)

In [7]:
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost,global_step=global_step)
    tf.summary.scalar('cost',cost)

In [9]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train_checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess,ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [10]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs',sess.graph)

In [13]:
for step in range(100):
    sess.run(train_op,feed_dict={X:x_data,Y:y_data})
    print('Step: %d, ' %sess.run(global_step),'Cost: %.3f' %sess.run(cost,feed_dict={X:x_data,Y:y_data}))
    summary = sess.run(merged,feed_dict={X:x_data,Y:y_data})
    writer.add_summary(summary,global_step=sess.run(global_step))

saver.save(sess, './model/dnn.ckpt',global_step=global_step)

Step: 3,  Cost: 1.036
Step: 4,  Cost: 0.992
Step: 5,  Cost: 0.950
Step: 6,  Cost: 0.913
Step: 7,  Cost: 0.879
Step: 8,  Cost: 0.847
Step: 9,  Cost: 0.817
Step: 10,  Cost: 0.787
Step: 11,  Cost: 0.760
Step: 12,  Cost: 0.735
Step: 13,  Cost: 0.712
Step: 14,  Cost: 0.691
Step: 15,  Cost: 0.672
Step: 16,  Cost: 0.656
Step: 17,  Cost: 0.641
Step: 18,  Cost: 0.629
Step: 19,  Cost: 0.617
Step: 20,  Cost: 0.607
Step: 21,  Cost: 0.599
Step: 22,  Cost: 0.591
Step: 23,  Cost: 0.585
Step: 24,  Cost: 0.580
Step: 25,  Cost: 0.575
Step: 26,  Cost: 0.572
Step: 27,  Cost: 0.569
Step: 28,  Cost: 0.566
Step: 29,  Cost: 0.564
Step: 30,  Cost: 0.562
Step: 31,  Cost: 0.560
Step: 32,  Cost: 0.559
Step: 33,  Cost: 0.558
Step: 34,  Cost: 0.557
Step: 35,  Cost: 0.556
Step: 36,  Cost: 0.555
Step: 37,  Cost: 0.555
Step: 38,  Cost: 0.554
Step: 39,  Cost: 0.554
Step: 40,  Cost: 0.553
Step: 41,  Cost: 0.553
Step: 42,  Cost: 0.553
Step: 43,  Cost: 0.553
Step: 44,  Cost: 0.552
Step: 45,  Cost: 0.552
Step: 46,  Cost: 0

'./model/dnn.ckpt-102'

In [14]:
prediction = tf.argmax(model,1)
target = tf.argmax(Y,1)
print('pre:',sess.run(prediction,feed_dict={X:x_data}))
print('tar:',sess.run(target,feed_dict={Y:y_data}))

pre: [0 1 2 0 0 2]
tar: [0 1 2 0 0 2]
